In [1]:
import requests
import json
import pandas as pd
import geopandas as gpd 
import matplotlib.pyplot as plt 
import numpy as np

#helper function to get responses of the api and store each layer in a dictionary
def get_responses(*args):
    list_of_responses={}
    for parm in args:
        url = f"https://emotional.byteroad.net/collections/{parm}/items?f=json&lang=en-US&limit=10000&skipGeometry=false&offset=0"
        response = requests.get(url)
        
        if response.status_code == 200:
            data_dict = json.loads(response.content)
            list_of_responses[parm]=data_dict
        else:
            print("Error: API request unsuccessful.")
    return list_of_responses

In [2]:
from branca.element import MacroElement
from jinja2 import Template

class BindColormap(MacroElement):
    def __init__(self, layer, colormap):
        super(BindColormap, self).__init__()
        self.layer = layer
        self.colormap = colormap
        self._template = Template(u"""
        {% macro script(this, kwargs) %}
            {{this.colormap.get_name()}}.svg[0][0].style.display = 'block';
            {{this._parent.get_name()}}.on('layeradd', function (eventLayer) {
                if (eventLayer.layer == {{this.layer.get_name()}}) {
                    {{this.colormap.get_name()}}.svg[0][0].style.display = 'block';
                }});
            {{this._parent.get_name()}}.on('layerremove', function (eventLayer) {
                if (eventLayer.layer == {{this.layer.get_name()}}) {
                    {{this.colormap.get_name()}}.svg[0][0].style.display = 'none';
                }});
        {% endmacro %}
        """)    

In [3]:
import folium
from folium import plugins
from folium import Map, FeatureGroup, Marker, LayerControl
import branca.colormap as cm
import branca

#dynamic legend values
column_names = ['pos_tweets','life_bir_r', 'mort_r','pat_diabe', 'pat_alcoh','pat_tobac', 'pat_obesi', 'pat_hyper', 'pat_demen', 'pat_anxie', 'pat_depre']

data = {column_name: [] for column_name in column_names}

responses = [
    get_responses('lisbon_positive_tweets_density')['lisbon_positive_tweets_density']['features'],
    get_responses('lisbon_life_births_rate')['lisbon_life_births_rate']['features'],
    get_responses('lisbon_mortality_rate')['lisbon_mortality_rate']['features'],
    get_responses('lisbon_patients_diabetes_mellitus')['lisbon_patients_diabetes_mellitus']['features'],
    get_responses('lisbon_patients_chronic_alcohol_abuse')['lisbon_patients_chronic_alcohol_abuse']['features'],
    get_responses('lisbon_patients_tobacco_abuse')['lisbon_patients_tobacco_abuse']['features'],
    get_responses('lisbon_patients_obesity')['lisbon_patients_obesity']['features'],
    get_responses('lisbon_patients_hypertension')['lisbon_patients_hypertension']['features'],
    get_responses('lisbon_patients_dementia')['lisbon_patients_dementia']['features'],
    get_responses('lisbon_patients_anxiety_disorder')['lisbon_patients_anxiety_disorder']['features'],
    get_responses('lisbon_patients_depressive_disorder')['lisbon_patients_depressive_disorder']['features']
]

for response_data in responses:
    for feature in response_data:
        properties = feature['properties']
        for column_name in column_names:
            value = properties.get(column_name, None)
            data[column_name].append(value)
        
df = pd.DataFrame(data)
#df.to_csv('df.csv')

#print(df)

min_values = df.min(skipna=True)
max_values = df.max(skipna=True)

num_groups = 4
group_boundaries = {}
for column in df.columns:
    min_value = min_values[column]
    max_value = max_values[column]
    
    if min_value is None or max_value is None:
        continue  # Skip columns with missing or null values
        
    group_size = (max_value - min_value) / num_groups
    boundaries = np.arange(min_value, max_value + group_size, group_size)
    group_boundaries[column] = boundaries

# Print the group boundaries for each column
for column, boundaries in group_boundaries.items():
    print(f"Group boundaries for column '{column}':")
    print(boundaries)
    print()

Group boundaries for column 'pos_tweets':
[ 0.        18.5961329 37.1922658 55.7883987 74.3845316]

Group boundaries for column 'life_bir_r':
[ 7.46135287  8.84194403 10.22253518 11.60312633 12.98371749]

Group boundaries for column 'mort_r':
[ 5.27209365  9.11783979 12.96358593 16.80933207 20.65507821]

Group boundaries for column 'pat_diabe':
[0.         2.32539129 4.65078257 6.97617386 9.30156515]

Group boundaries for column 'pat_alcoh':
[0.         0.66415481 1.32830961 1.99246442 2.65661922]

Group boundaries for column 'pat_tobac':
[ 0.          4.76761846  9.53523691 14.30285537 19.07047383]

Group boundaries for column 'pat_obesi':
[ 0.          4.32494129  8.64988258 12.97482387 17.29976516]

Group boundaries for column 'pat_hyper':
[ 0.         22.86679524 45.73359047 68.60038571 91.46718094]

Group boundaries for column 'pat_demen':
[0.         1.10447825 2.20895649 3.31343474 4.41791299]

Group boundaries for column 'pat_anxie':
[0.         2.16982674 4.33965347 6.50948021

In [4]:
def hexmap(responses, column_name, color, feature_group):
    for response in responses:
        for feature in response:
            properties = feature['properties']
            if column_name in properties:
                value = properties[column_name]
                geoj = folium.GeoJson(feature, style_function=color)
                folium.features.GeoJsonPopup(fields=[column_name], aliases=['Mean Value'], labels=True, localize=True).add_to(geoj)
                feature_group.add_child(geoj)

#pos_tweets

color_pos_tweets = lambda x: {
    'fillColor': '#FFFFFF' if x['properties']['pos_tweets'] < 0 else 
    '#0000FF' if 0 <= x['properties']['pos_tweets'] < 18.60 else 
    '#ADD8E6' if 18.60 <= x['properties']['pos_tweets'] < 37.19 else 
    '#90EE90' if 37.19 <= x['properties']['pos_tweets'] < 55.79 else
    '#008000', 'color': 'white', 'weight': 1, 'fillOpacity': 0.7
}

fg1 = folium.FeatureGroup(name='Positive tweetse', overlay=True, control=False)
hexmap(responses, 'pos_tweets', color_pos_tweets, fg1)
scale_fg1 = branca.colormap.StepColormap(['#0000FF','#ADD8E6','#90EE90','#008000'],index=[0,18.60,37.19,55.79,74.38], vmin=0, vmax=74.38, caption='Positive tweets')


#l_birth_ra

color_birth = lambda x: {
    'fillColor': '#FFFFFF' if x['properties']['life_bir_r'] < 7.46 else 
    '#FFE6CC' if 7.46 <= x['properties']['life_bir_r'] < 8.84 else 
    '#FFD9A3' if 8.84 <= x['properties']['life_bir_r'] < 10.22 else 
    '#FFCC7A' if 10.22 <= x['properties']['life_bir_r'] < 11.60 else
    '#FFBF51', 'color': 'white', 'weight': 1, 'fillOpacity': 0.7
}

fg2 = folium.FeatureGroup(name='Life births rate', overlay=True, control=False)
hexmap(responses, 'life_bir_r', color_birth, fg2)
scale_fg2 = branca.colormap.StepColormap(['#FFE6CC','#FFD9A3','#FFCC7A','#FFBF51'],index=[7.46,8.84,10.22,11.60,12.98], vmin=7.46, vmax=12.98, caption='Life births rate')

#morta_rat

color_morta = lambda x: {
    'fillColor': '#FFFFFF' if x['properties']['mort_r'] < 5.27 else 
    '#00FF00' if 5.27 <= x['properties']['mort_r'] < 9.12 else 
    '#FFFF00' if 9.12 <= x['properties']['mort_r'] < 12.96 else 
    '#FFA500' if 12.96 <= x['properties']['mort_r'] < 16.81 else
    '#FF0000', 'color': 'white', 'weight': 1, 'fillOpacity': 0.7
}

fg3 = folium.FeatureGroup(name='Mortality rate', overlay=True, control=False)
hexmap(responses, 'mort_r', color_morta, fg3)
scale_fg3 = branca.colormap.StepColormap(['#00FF00','#FFFF00','#FFA500','#FF0000'],index=[5.27,9.12,12.96,16.81,20.66], vmin=5.27, vmax=20.66, caption='Mortality rate')


#pp_diabete

color_dia = lambda x: {
    'fillColor': '#FFFFFF' if x['properties']['pat_diabe'] < 0 else 
    '#FFC6C6' if 0 <= x['properties']['pat_diabe'] < 2.33 else 
    '#FF8C8C' if 2.33 <= x['properties']['pat_diabe'] < 4.65 else 
    '#FF5252' if 4.65 <= x['properties']['pat_diabe'] < 6.98 else
    '#FF0000', 'color': 'white', 'weight': 1, 'fillOpacity': 0.7
}

fg4 = folium.FeatureGroup(name='Patients with Diabetes Mellitus', overlay=True, control=False)
hexmap(responses, 'pat_diabe', color_dia, fg4)
scale_fg4 = branca.colormap.StepColormap(['#FFC6C6','#FF8C8C','#FF5252','#FF0000'],index=[0,2.33,4.65,6.98,9.30], vmin=0, vmax=9.30, caption='Patients with Diabetes Mellitus')

#pp_alcohol

color_alcohol = lambda x: {
    'fillColor': '#FFFFFF' if x['properties']['pat_alcoh'] < 0 else 
    '#FFD9B3' if 0 <= x['properties']['pat_alcoh'] < 0.66 else 
    '#FFB380' if 0.66 <= x['properties']['pat_alcoh'] < 1.33 else 
    '#FF8C4D' if 1.33 <= x['properties']['pat_alcoh'] < 1.99 else
    '#FF661A', 'color': 'white', 'weight': 1, 'fillOpacity': 0.7
}

fg5 = folium.FeatureGroup(name='Patients with chronic alcohol abuse', overlay=True, control=False)
hexmap(responses, 'pat_alcoh', color_alcohol, fg5)
scale_fg5 = branca.colormap.StepColormap(['#FFD9B3','#FFB380','#FF8C4D','#FF661A'],index=[0,0.66,1.33,1.99,2.66], vmin=0, vmax=2.66, caption='Patients with chronic alcohol abuse')

#pp_tobacco

color_tobacco = lambda x: {
    'fillColor': '#FFFFFF' if x['properties']['pat_tobac'] < 0 else 
    '#FFD9B3' if 0 <= x['properties']['pat_tobac'] < 4.77 else 
    '#FFB380' if 4.77 <= x['properties']['pat_tobac'] < 9.54 else 
    '#FF8C4D' if 9.54 <= x['properties']['pat_tobac'] < 14.30 else
    '#FF661A', 'color': 'white', 'weight': 1, 'fillOpacity': 0.7
}

fg6 = folium.FeatureGroup(name='Patients with tobacco abuse', overlay=True, control=False)
hexmap(responses, 'pat_tobac', color_tobacco, fg6)
scale_fg6 = branca.colormap.StepColormap(['#FFD9B3','#FFB380','#FF8C4D','#FF661A'],index=[0,4.77,9.54,14.30,19.07], vmin=0, vmax=19.07, caption='Patients with tobacco abuse')


#pp_obesity

color_obesity = lambda x: {
    'fillColor': '#FFFFFF' if x['properties']['pat_obesi'] < 0 else 
    '#FFFF99' if 0 <= x['properties']['pat_obesi'] < 4.32 else 
    '#FF6347' if 4.32 <= x['properties']['pat_obesi'] < 8.65 else 
    '#AA33AA' if 8.65 <= x['properties']['pat_obesi'] < 12.97 else
    '#4B0082', 'color': 'white', 'weight': 1, 'fillOpacity': 0.7
}

fg7 = folium.FeatureGroup(name='Patients with obesity', overlay=True, control=False)
hexmap(responses, 'pat_obesi', color_obesity, fg7)
scale_fg7 = branca.colormap.StepColormap(['#FFFF99','#FF6347','#AA33AA','#4B0082'],index=[0,4.32,8.65,12.97,17.30], vmin=0, vmax=17.30, caption='Patients with obesity')


#pp_hyperte

color_hyp = lambda x: {
    'fillColor': '#FFFFFF' if x['properties']['pat_hyper'] < 0 else 
    '#FFFF99' if 0 <= x['properties']['pat_hyper'] < 22.87 else 
    '#FF6347' if 22.87 <= x['properties']['pat_hyper'] < 45.73 else 
    '#AA33AA' if 45.73 <= x['properties']['pat_hyper'] < 68.60 else
    '#4B0082', 'color': 'white', 'weight': 1, 'fillOpacity': 0.7
}

fg8 = folium.FeatureGroup(name='Patients with hypertension', overlay=True, control=False)
hexmap(responses, 'pat_hyper', color_hyp, fg8)
scale_fg8 = branca.colormap.StepColormap(['#FFFF99','#FF6347','#AA33AA','#4B0082'],index=[0,22.87,45.73,68.60,91.47], vmin=0, vmax=91.47, caption='Patients with hypertension')


#pp_dementi

color_dementi = lambda x: {
    'fillColor': '#FFFFFF' if x['properties']['pat_demen'] < 0 else 
    '#CCFFCC' if 0 <= x['properties']['pat_demen'] < 1.10 else 
    '#20B2AA' if 1.10 <= x['properties']['pat_demen'] < 2.21 else 
    '#007F92' if 2.21 <= x['properties']['pat_demen'] < 3.31 else
    '#20002C', 'color': 'white', 'weight': 1, 'fillOpacity': 0.7
}

fg9 = folium.FeatureGroup(name='Patients diagnosed with dementia', overlay=True, control=False)
hexmap(responses, 'pat_demen', color_dementi, fg9)
scale_fg9 = branca.colormap.StepColormap(['#CCFFCC','#20B2AA','#007F92','#20002C'],index=[0,1.10,2.21,3.31,4.42], vmin=0, vmax=4.42, caption='Patients diagnosed with dementia')


#pp_anxi_di

color_anxi = lambda x: {
    'fillColor': '#FFFFFF' if x['properties']['pat_anxie'] < 0 else 
    '#CCFFCC' if 0 <= x['properties']['pat_anxie'] < 2.17 else 
    '#20B2AA' if 2.17 <= x['properties']['pat_anxie'] < 4.34 else 
    '#007F92' if 4.34 <= x['properties']['pat_anxie'] < 6.51 else
    '#20002C', 'color': 'white', 'weight': 1, 'fillOpacity': 0.7
}

fg10 = folium.FeatureGroup(name='Patients diagnosed with anxiety disorder', overlay=True, control=False)
hexmap(responses, 'pat_anxie', color_anxi, fg10)
scale_fg10 = branca.colormap.StepColormap(['#CCFFCC','#20B2AA','#007F92','#20002C'],index=[0,2.17,4.34,6.51,8.68], vmin=0, vmax=8.68, caption='Patients diagnosed with anxiety disorder')


#pp_depress

color_depress = lambda x: {
    'fillColor': '#FFFFFF' if x['properties']['pat_depre'] < 0 else 
    '#CCFFCC' if 0 <= x['properties']['pat_depre'] < 3.37 else 
    '#20B2AA' if 3.37 <= x['properties']['pat_depre'] < 6.75 else 
    '#007F92' if 6.75 <= x['properties']['pat_depre'] < 10.12 else
    '#20002C', 'color': 'white', 'weight': 1, 'fillOpacity': 0.7
}  

fg11 = folium.FeatureGroup(name='Patients diagnosed with depressive disorder',overlay=True, control=False)
hexmap(responses, 'pat_depre', color_depress, fg11)
scale_fg11 = branca.colormap.StepColormap(['#CCFFCC','#20B2AA','#007F92','#20002C'],index=[0,3.37,6.75,10.12,13.49], vmin=0, vmax=13.49, caption='Patients diagnosed with depressive disorder')

In [5]:
# Add each layer to the map and save it as a separate HTML file

for i in range(1, 12):
    # Create a new map object for each iteration
    m = folium.Map(location=[38.736946, -9.142685], control_scale=True, zoom_start=12, tiles=None)
    
    base_map1 = folium.FeatureGroup(name='OpenStreetMap', overlay=False)
    folium.TileLayer(tiles='OpenStreetMap').add_to(base_map1)
    # Add satellite imagery basemap
    base_map2 = folium.FeatureGroup(name="Esri World Imagery", overlay=False)
    folium.TileLayer(
        tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr='Esri',
        name='Esri Satellite'
    ).add_to(base_map2)

    base_map1.add_to(m)
    base_map2.add_to(m)
    
    fg_name = "fg" + str(i)
    fg = globals()[fg_name]
    m.add_child(fg)
    
    scale_name = "scale_fg" + str(i)
    scale = globals()[scale_name]
    m.add_child(scale)
    m.add_child(BindColormap(fg, scale))

    # Add LayerControl to toggle between basemaps
    folium.LayerControl(position='bottomleft').add_to(m)
    
    map_fname = f'Output_Lisbon_Layer{i}.html'
    m.save(map_fname)